# Logistic Regression Code Along
This is a code along of the famous titanic dataset, its always nice to start off with this dataset because it is an example you will find across pretty much every data analysis language.

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('myproj').getOrCreate()

In [3]:
data = spark.read.csv('../titanic.csv',inferSchema=True,header=True)

In [4]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [5]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [6]:
# 필요한 컬럼만 가져옴
my_cols = data.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [7]:
# 결측값은 삭제했음
my_final_data = my_cols.na.drop()

### Working with Categorical Columns

Let's break this down into multiple steps to make it all clear.

In [8]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [9]:
# 두 단계를 거쳐 원-핫 인코딩한다.
# 1. 문자열을 숫자로
# 2, 바뀐 숫자들을 원-핫 인코딩
# 결과는 벡터 형식으로 출력된다.
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [10]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [11]:
assembler = VectorAssembler(inputCols=['Pclass',
 'SexVec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'EmbarkVec'],outputCol='features')

In [12]:
from pyspark.ml.classification import LogisticRegression

## Pipelines 

Let's see an example of how to use pipelines (we'll get a lot more practice with these later!)

각 작업의 진행을 위해 단계를 생성하는 역할을 한다.

In [13]:
from pyspark.ml import Pipeline

In [14]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [15]:
# 파이프라인은 작업에 필요한 요소를 모두 포함하는 'stages'라는 인자를 갖는다.
# (인자에는 앞에서 만들어 놓은 객체를 저장한 변수를 리스트에 나열한다.)
# 변환을 위해 단계를 설계하고 집합시킨 다음 모델에게 넘겨주는 것이라 볼 수 있다.
# (리스트로 나열하는 것이 단계를 설계하는 것으로 보이고, 집합시키고 모델에
# 넘겨주는 것은 파이프라인 객체가 하는 것으로 보인다.)
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                           gender_encoder,embark_encoder,
                           assembler,log_reg_titanic])

In [16]:
train_titanic_data, test_titanic_data = my_final_data.randomSplit([0.7,.3])

In [17]:
# 파이프라인 객체의 장점은 일반적인 모델처럼 사용할 수 있다는 것이다.
fit_model = pipeline.fit(train_titanic_data)

In [18]:
# 테스트 데이터를 변환
# 결과물을 테스트 데이터 셋에서 시험해보는거라함
# (이전까지는 fit -> evaluate 였고, 배포단계에서 transform을 사용했는데.. 헷갈린다.)
# (evaluate은 LinearRegressionSummary객체를 반환한다. 속성으로 평가지표를 갖는다.)
# (accuracy, weightedPrecision, weightedRecall, ... )
# (여태 사용한 predictions도 있는데 Dataframe outputted by the model's transform method을 의미한다. )
results = fit_model.transform(test_titanic_data)

In [19]:
results.show()
# test_titanic_data에 파이프라인이 모두 적용되고 예측값까지 추가됨
# 그냥 이걸 transform이 하는일이라고 생각하자

+--------+------+------+----+-----+-----+--------+--------+--------+-----------+-------------+-------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|    Fare|Embarked|SexIndex|EmbarkIndex|       SexVec|    EmbarkVec|            features|       rawPrediction|         probability|prediction|
+--------+------+------+----+-----+-----+--------+--------+--------+-----------+-------------+-------------+--------------------+--------------------+--------------------+----------+
|       0|     1|female| 2.0|    1|    2|  151.55|       S|     1.0|        0.0|    (1,[],[])|(2,[0],[1.0])|[1.0,0.0,2.0,1.0,...|[-4.0662102912122...|[0.01685332603877...|       1.0|
|       0|     1|female|50.0|    0|    0| 28.7125|       C|     1.0|        1.0|    (1,[],[])|(2,[1],[1.0])|(8,[0,2,5,7],[1.0...|[-2.5347019977799...|[0.07346096513819...|       1.0|
|       0|     1|  male|18.0|    1|    0|   108.9|       C|     0.0|        1.0|(1,[0

In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [21]:
# results.show()에서 prediction 컬럼이 추가된 것을 보았다.
# 따라서 rawPredictionCol에 'prediction'을 넣었다.
# transform하면 항상 prediction 컬럼이 생성된다고 함

my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Survived')

In [22]:
AUC = my_eval.evaluate(results)

In [23]:
AUC

0.7599624060150376

## Great Job!